<a href="https://colab.research.google.com/github/andrey-varna/OSR_passport/blob/main/Stazhirovka_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''скачиваю clone YOLOv5 ''' 
!git clone https://github.com/ultralytics/yolov5                                # Скачиваю clone с репрозитория
%cd yolov5
%pip install -qr requirements.txt                                               # Устанавливаю зависимости
%pip install -q roboflow                                                        # Устанавливаю roboflow 
''' устанавливаю дополнительные модули '''
import os                                                                       # Модуль работы с файловой системой
from IPython.display import Image, clear_output                                 # Модуль для вывода изображений

Cloning into 'yolov5'...
remote: Enumerating objects: 15646, done.
remote: Counting objects: 100% (253/253), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 15646 (delta 127), reused 134 (delta 64), pack-reused 15393
Receiving objects: 100% (15646/15646), 14.59 MiB | 21.20 MiB/s, done.
Resolving deltas: 100% (10656/10656), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
!pip install easyocr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72


In [3]:
import cv2
import numpy as np
import math
import torch
import pandas as pd
import keras
import pickle
from keras.models import load_model
import tensorflow as tf

class Passport:
    def __init__(self):
        self.ModelFolder=''#"OCRModels/"
        self.Model1=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_numbers_v2.h5')
        self.Model2=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_v2.h5')
        self.model_round = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/rotation.pt', source='local')
        self.model_detect = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/detect.pt', source='local')
        self.Model1_2_img_width = 1312  #1148  # 750
        self.Model1_2_img_height = 64  #164   # 64
        with open(self.ModelFolder+"OCRModels/characters_letters.pickle", 'rb') as f:
            Model1_2_characters= pickle.load(f)
        char_to_num = keras.layers.StringLookup(vocabulary=list(Model1_2_characters), mask_token=None)
        self.num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
        self.MaxLenLabel_1_2=80

		
    def encode_single_sample(self,img,ConvertionType=0):#type1=0 - Numbers  type=1 - Text
        # 1. Read image
        #img = tf.io.read_file(img_path)
        # 2. Decode and convert to grayscale
        #img = tf.io.decode_jpeg(img, channels=1)
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # 3. Convert to float32 in [0, 1] range
        img = np.expand_dims(img,axis=-1)
	
        img1=img.copy()
        img1 = tf.image.convert_image_dtype(img1, tf.float32)
        #print(type(img1),img1.shape)
        # 4. Resize to the desired size
        # h, w, ch = img.shape.as_list()
        # img_width = int(w * img_height / h)
        img1 = tf.image.resize(img1, [self.Model1_2_img_height, 740])
        #print(type(img1),img1.shape)
        # 4.1
        # h, w, ch = img.shape.as_list()
        # ones = tf.ones((h, (1312 - w), ch), dtype=tf.float32)
        ones = tf.ones((self.Model1_2_img_height, (self.Model1_2_img_width - 740), 1), dtype=tf.float32)
        img1 = tf.concat([img1, ones], 1)
        # 5. Transpose the image because we want the time
        # dimension to correspond to the width of the image.
        img1 = tf.transpose(img1, perm=[1, 0, 2])
        # 6. Map the characters in label to numbers
        #label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
	
        if(ConvertionType==0):
            Prediction=self.Model1.predict(np.expand_dims(img1,0),verbose=0)#np.expand_dims(ImArr,0)
        elif(ConvertionType==1):
            Prediction=self.Model2.predict(np.expand_dims(img1,0),verbose=0)
			
        input_len = np.ones(Prediction.shape[0]) * Prediction.shape[1]
        # Use greedy search. For complex tasks, you can use beam search
        results = keras.backend.ctc_decode(Prediction, input_length=input_len, greedy=True)[0][0][:,:self.MaxLenLabel_1_2]
        output_text=tf.strings.reduce_join(self.num_to_char(results[0])).numpy().decode("utf-8")
        output_text=output_text.replace("[UNK]","")
        #print(output_text)
        BreaksEnd=False
        BreaksEndPos=0
        for i in range(-1,-len(output_text),-1):
            if(output_text[i]!=" " and BreaksEndPos==0):
                BreaksEnd=True
                BreaksEndPos=i
                break
            #print(BreaksEndPos)
        if(BreaksEndPos<-1):
            output_text=output_text[:BreaksEndPos+1]
        #print("output_text",output_text)
        return output_text
		
		
    def zero(self,n):
        return n * (n > 0)

    def rotate_image(self, mat, angle):
        height, width = mat.shape[:2]
        image_center = (width / 2, height / 2)
        rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)
        abs_cos = abs(rotation_mat[0, 0])
        abs_sin = abs(rotation_mat[0, 1])
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)
        rotation_mat[0, 2] += bound_w / 2 - image_center[0]
        rotation_mat[1, 2] += bound_h / 2 - image_center[1]
        return cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))

    def get_angle_rotation(self, centre, point, target_angle):
        new_point =(point[0] - centre[0], point[1] - centre[1])
        a,b = new_point[0], new_point[1]
        res = math.atan2(b,a)
        if (res < 0) :
              res += 2 * math.pi
        return (math.degrees(res)+target_angle) % 360

    def result(self,img):
        return self.model_round(img)

    def get_image_after_rotation(self, img):
        results = self.result(img)
        pd = results.pandas().xyxy[0]
        pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)
        pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)
        tmp = pd.loc[pd['name']=='niz']
        N, V = None, None
        for index, row in tmp.iterrows():
            N = (row['centre_x'], row['centre_y'])
            break
        tmp = pd.loc[pd['name']=='verh']
        for index, row in tmp.iterrows():
            V = (row['centre_x'], row['centre_y'])
            break
        if N == None or V == None:
            return img

        angle = self.get_angle_rotation(N, V, 90)
        img = self.rotate_image(img, angle)
        return img

    def crop_img(self, img):
        #print(img.size)
        results = self.result(img)
        #print(results.pandas())
        pd = results.pandas().xyxy[0]
        x1 =int(pd.xmin.min())
        x2 = int(pd.xmax.max())
        y1 = int(pd.ymin.min())
        y2 = int(pd.ymax.max())
        img = img[y1:y2,x1:x2]
        return img

    def get_crop(self,file):
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        image = self.get_image_after_rotation(image)
        image = self.get_image_after_rotation(image)
        image = self.crop_img(image)
        return image

    def yolo_5(self, img):
        results = self.model_detect(img)
        df = results.pandas().xyxy[0]
        df = df.drop(np.where(df['confidence'] < 0.7)[0])
        # print(df)
        ob = pd.DataFrame()
        ob['class'] = df['name']
        ob['x'] = df['xmin']
        ob['y'] = df['ymin']
        ob['w'] = df['xmax']-df['xmin']
        ob['h'] = df['ymax']-df['ymin']
        oblasty = ob.values.tolist()
        return img, oblasty

    def oblasty_yolo_5(self, image, box):
        oblasty = {}
        iss = 0
        plac = 0
        spissok = sorted(box, reverse=False, key=lambda x: x[2])
        for l in spissok:
            cat = l[0]
            y = int(l[2])
            x = int(l[1])
            h = int(l[4])
            w = int(l[3])
            ob = ''
            if ('signature' in cat) or ('photograph' in cat):
                pass
            else:
                if 'issued_by_whom' in cat:
                    ob = cat + '_' + str(iss)
                    iss += 1
                elif 'place_of_birth' in cat:
                    ob = cat + '_' + str(plac)
                    plac += 1
                elif 'series' not in cat:
                    ob = cat
                if ob:
                    oblasty[ob] = image[self.zero(y - math.ceil(h * 0.07)):y + math.ceil(h * 1.1),
                                  self.zero(x - math.ceil(w * 0.1)):x + math.ceil(w * 1.1)]
                if 'series' in cat:
                    ob = cat
                    cropped = image[self.zero(y - math.ceil(h * 0.1)):y + math.ceil(h * 1.1),
                              self.zero(x - math.ceil(w * 0.03)):x + math.ceil(w * 1.03)]
                    oblasty[ob] = cv2.rotate(cropped, cv2.ROTATE_90_COUNTERCLOCKWISE) #self.rotate_image(cropped, 90)
        return oblasty

    def recognition_slovar(self, oblasty):
        data = {}
        data['pasport'] = []
        d = {}
        issued_by_whom = ''
        series_and_number = ''
        place_of_birth = ''
        ver = 0
        for i, v in oblasty.items():
            image = cv2.cvtColor(v, cv2.COLOR_BGR2RGB)
            # if image.shape[0] <=23:
            #     image = self.resiz(image)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            # Для каждого класса устанавливаем свои ограничения на распознания классов
            #print(i,np.array(image).shape)
            if 'date' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'code' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'series' in i:
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'surname' in i or 'name' in i or 'patronymic' in i:
                result = self.encode_single_sample(image, ConvertionType=1)
            elif 'gender' in i:
                result = self.encode_single_sample(image, ConvertionType=1)
            else:
                result = self.encode_single_sample(image,ConvertionType=1)
            pole = result#''
            """
            for k in range(len(result)):
                if result[k][2] * 100 >= 25:
                    if str(result[k][1]).isnumeric():
                        if result[k][2] * 100 >= 70:
                            pole = pole + ' ' + str(result[k][1])
                    else:
                            pole = pole + ' ' + str(result[k][1])
            """
            if pole:
                pole = pole.strip()
                if 'issued_by_whom' in i:
                    issued_by_whom = issued_by_whom + pole + ' '
                if 'place_of_birth' in i:
                    place_of_birth = place_of_birth + pole + ' '
                if 'series_and_number' in i:
                    # print(pole)
                    """
                    if len(pole) >= 10:
                        if ver < result[k][2]:
                            series_and_number = pole
                    """
                    series_and_number = pole
                    
                if 'issued_by_whom' in i or 'place_of_birth' in i or 'series_and_number' in i:
                    pass
                elif 'date' in i:
                    pole = pole.replace('.', '').replace(' ', '').replace('-', '')
                    pole = pole[:2] + '.' + pole[2:4] + '.' + pole[4:8]
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                elif 'cod' in i:
                    pole = pole.replace(' . ', '').replace(' ', '').replace('-', '')
                    pole = pole[:3] + '-' + pole[3:6]
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                elif 'gender' in i:
                    pole1 = ''
                    if 'Е' in pole.upper() or 'Н' in pole.upper() or 'ЕН' in pole.upper():
                        pole1 = 'ЖЕН.'
                    if 'У' in pole.upper() or 'М' in pole.upper() or 'УЖ' in pole.upper():
                        pole1 = 'МУЖ.'
                    pole = pole1
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                else:
                    d[i.split('.', 1)[0]] = pole.replace('  ', ' ').upper().strip()

        place_of_birth = place_of_birth.upper()
        issued_by_whom = issued_by_whom.upper()
        
        if place_of_birth[:2] == 'C ':
            place_of_birth = place_of_birth.replace('С ', ' С. ')
        if issued_by_whom[:2] == 'C ':
            issued_by_whom = issued_by_whom.replace('С ', ' С. ')

        place_of_birth = place_of_birth.replace('ГОР ', 'ГОР. ')\
                                        .replace(' С ', ' С. ')\
                                        .replace(' Г ', ' Г. ')\
                                        .replace('ОБЛ ', 'ОБЛ. ')\
                                        .replace('ПОС ', 'ПОС. ')\
                                        .replace('ДЕР ', 'ДЕР. ')\
                                        .replace(' . ', '. ')\
                                        .replace(' .', '.')\
                                        .replace('  ', ' ')\
                                        .replace('..', '.')\
                                        .replace('.', '. ')\
                                        .replace('  ', ' ')\
                                        .replace('--', '-')

        issued_by_whom = issued_by_whom.replace('ГОР ', 'ГОР. ')\
                                        .replace(' С ', ' С. ')\
                                        .replace(' Г ', ' Г. ')\
                                        .replace('ОБЛ ', 'ОБЛ. ')\
                                        .replace('ПОС ', 'ПОС. ')\
                                        .replace('ДЕР ', 'ДЕР. ')\
                                        .replace(' . ', '. ')\
                                        .replace(' .', '.')\
                                        .replace('  ', ' ')\
                                        .replace('..', '.')\
                                        .replace('.', '. ')\
                                        .replace('  ', ' ')\
                                        .replace('--', '-')
        if series_and_number:
            series_and_number = series_and_number.replace(' ', '')
            if len(series_and_number) == 10:
                series_and_number = series_and_number[:2] + ' ' + series_and_number[2:4] + ' ' + series_and_number[4:10]
            else:
                series_and_number = 'поле распознано не полностью' + series_and_number
        else:
            series_and_number = ''
        d['issued_by_whom'] = issued_by_whom.strip()
        d['place_of_birth'] = place_of_birth.strip()
        d['series_and_number'] = series_and_number.strip()
        data['pasport'].append(d)
        return data['pasport']

    def detect_passport(self,photo):
        pole = ['date_of_birth','date_of_issue','first_name','gender','issued_by_whom',
                'patronymic','place_of_birth','series_and_number','surname','unit_code']

        croped = self.get_crop(photo)
        if croped != '':
            img, detect = self.yolo_5(croped)
            obl = self.oblasty_yolo_5(img, detect)
            rec = self.recognition_slovar(obl)
            key = list(rec[0].keys())
            value = list(rec[0].values())
            if set(key) == set(pole):
                if '' in value:
                    flag = 1

                else:
                    flag = 0
                    if len(rec[0]['date_of_birth']) != 10 or len(rec[0]['date_of_issue']) != 10 or len(rec[0]['series_and_number']) != 12 or len(rec[0]['unit_code']) != 7:
                        flag = 1
            else:
                flag = 1

            if flag == 1:
                rec =rec[0]# {}
            else:
                rec = rec[0]
            return rec, flag
        else:
            return {}, 1

		

class INN(Passport):
    """
    def __init__(self):
        self.reader = easyocr.Reader(['ru'],
                                     model_storage_directory='EasyOCR/model',
                                     user_network_directory='EasyOCR/user_network',
                                     recog_network='custom_example',
                                     gpu=False)

        self.model_round_inn = torch.hub.load('yolov5_master', 'custom', path='yolo5/inn_rotation.pt', source='local')
        self.model_detect_inn = torch.hub.load('yolov5_master', 'custom', path='yolo5/fio_INN.pt', source='local')
        self.model_numbers = torch.hub.load('yolov5_master', 'custom', path='yolo5/best_X_INN.pt', source='local')
    """
    def __init__(self):
        self.ModelFolder=''#"OCRModels/"
        self.Model1=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_numbers_v2.h5')
        self.Model2=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_v2.h5')
        self.model_round_inn = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/inn_rotation.pt', source='local')
        self.model_detect_inn = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/fio_INN.pt', source='local')
        self.model_numbers = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/best_X_INN.pt', source='local')
        self.Model1_2_img_width = 1312  #1148  # 750
        self.Model1_2_img_height = 64  #164   # 64
        with open(self.ModelFolder+"OCRModels/characters_letters.pickle", 'rb') as f: Model1_2_characters= pickle.load(f)
        char_to_num = keras.layers.StringLookup(vocabulary=list(Model1_2_characters), mask_token=None)
        self.num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
        self.MaxLenLabel_1_2=80

    def encode_single_sample(self,img,ConvertionType=0):#type1=0 - Numbers  type=1 - Text
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(img,axis=-1)
	
        img1 = img.copy()
        img1 = tf.image.convert_image_dtype(img1, tf.float32)
        img1 = tf.image.resize(img1, [self.Model1_2_img_height, 740])
        ones = tf.ones((self.Model1_2_img_height, (self.Model1_2_img_width - 740), 1), dtype=tf.float32)
        img1 = tf.concat([img1, ones], 1)
        img1 = tf.transpose(img1, perm=[1, 0, 2])
        if(ConvertionType==0):
            Prediction=self.Model1.predict(np.expand_dims(img1,0),verbose=0)#np.expand_dims(ImArr,0)
        elif(ConvertionType==1):
            Prediction=self.Model2.predict(np.expand_dims(img1,0),verbose=0)
			
        input_len = np.ones(Prediction.shape[0]) * Prediction.shape[1]
        # Use greedy search. For complex tasks, you can use beam search
        results = keras.backend.ctc_decode(Prediction, input_length=input_len, greedy=True)[0][0][:,:self.MaxLenLabel_1_2]
        output_text=tf.strings.reduce_join(self.num_to_char(results[0])).numpy().decode("utf-8")
        output_text=output_text.replace("[UNK]","")
        #print(output_text)
        BreaksEnd=False
        BreaksEndPos=0
        for i in range(-1,-len(output_text),-1):
            if(output_text[i]!=" " and BreaksEndPos==0):
                BreaksEnd=True
                BreaksEndPos=i
                break
            #print(BreaksEndPos)
        if(BreaksEndPos<-1):
            output_text=output_text[:BreaksEndPos+1]
        #print("output_text",output_text)
        return output_text

    

    def result_inn(self, img):
        return self.model_round_inn(img)

    def get_angle_rotation_inn(self, centre, point, target_angle):
        new_point = (point[0] - centre[0], point[1] - centre[
            1])  # передвигаем центр системы координат в точку центр. у нее будет (0,0) ищем новые координаты у точки point
        a, b = new_point[0], new_point[1]
        res = math.atan2(b, a)  # ищем полярный угол у new_point
        if (res < 0):
            res += 2 * math.pi
        return (math.degrees(res) + target_angle) % 360  # возвращаем угол поворота для cv2

    def get_image_after_rotation_inn(self, img, point1, point2):
        results = self.result_inn(img)
        pd = results.pandas().xyxy[0]
        pd = pd.assign(centre_x=pd.xmin + (pd.xmax - pd.xmin) / 2)
        pd = pd.assign(centre_y=pd.ymin + (pd.ymax - pd.ymin) / 2)
        tmp = pd.loc[pd['name'] == point1]
        LEFT, RIGHT = None, None
        for index, row in tmp.iterrows():
            LEFT = (row['centre_x'], row['centre_y'])
            break
        # получим координаты верха, там где печать
        tmp = pd.loc[pd['name'] == point2]
        for index, row in tmp.iterrows():
            RIGHT = (row['centre_x'], row['centre_y'])
            break
        if LEFT == None or RIGHT == None:  # похоже там нет нужных нам строк
            return img

        angle = self.get_angle_rotation_inn(LEFT, RIGHT, 0)
        img = self.rotate_image(img, angle)  # вращаем той процедурой, что выше
        return img

    def get_crop_inn(self, file):
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # сразу в серый
        image = self.get_image_after_rotation_inn(image, point1='sv',
                                                  point2='vo')  # передаем в процедуру метки, котоыре будут использоваться для вращения первая - левая метка, вторая правая, будут ставиться в горизонт
        image = self.get_image_after_rotation_inn(image, point1='lev', point2='prav')
        return image

    def yolo_5_inn(self, img):
        results = self.model_detect_inn(img)
        df = results.pandas().xyxy[0]
        df = df.drop(np.where(df['confidence'] < 0.7)[0])
        ob = pd.DataFrame()
        ob['class'] = df['name']
        ob['x'] = df['xmin']
        ob['y'] = df['ymin']
        ob['w'] = df['xmax'] - df['xmin']
        ob['h'] = df['ymax'] - df['ymin']
        oblasty = ob.values.tolist()
        return img, oblasty

    def oblasty_yolo_5_inn(self, image, box):
        oblasty = {}
        spissok = sorted(box, reverse=False, key=lambda x: x[2])
        for l in spissok:
            cat = l[0]
            y = int(l[2])
            x = int(l[1])
            h = int(l[4])
            w = int(l[3])
            ob = cat
            oblasty[ob] = image[self.zero(y - math.ceil(h * 0.1)):y + math.ceil(h * 1.1),
                          self.zero(x - math.ceil(w * 0.03)):x + math.ceil(w * 1.03)]
        return oblasty

    def Intersection(self, boxA, boxB):
        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        return interArea

    def IoU(self, boxA, boxB):
        # compute the area of intersection rectangle
        interArea = self.Intersection(boxA, boxB)

        # compute the area of both the prediction and ground-truth rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

        # calculation iou
        iou = interArea / float(boxAArea + boxBArea - interArea)

        # return the intersection over union value
        return iou

    def numbers_detect(self, img):

        image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        result = self.model_numbers(image)

        final_res = result.pandas().xyxy[0]
        result = result.pandas().xyxy[0]

        # delete intersection objects
        for obj1 in range(len(result)):
            for obj2 in range(len(result)):

                boxA = result.iloc[obj1, :4].values
                boxB = result.iloc[obj2, :4].values

                if boxA.all() != boxB.all():
                    if self.IoU(boxA, boxB) > 0.2:
                        if result.iloc[obj1, 4] > result.iloc[obj2, 4]:
                            final_res = final_res[final_res.xmin != result.iloc[obj2, 0]]

                        else:
                            final_res = final_res[final_res.xmin != result.iloc[obj1, 0]]
        return final_res

    def recognition_slovar_inn(self, oblasty):
        data = {}
        data['inn'] = []
        d = {}
        for i, v in oblasty.items():
            image = cv2.cvtColor(v, cv2.COLOR_BGR2RGB)

            # plt.imshow(image)
            # plt.show()

            if 'inn' in i:
                """
                numbs = self.numbers_detect(image).sort_values(by=['xmin']).iloc[:, 5].values
                res_numbs = ''.join(str(e) for e in numbs)

                result = [([], f'{res_numbs}', 0.0)]
                """
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'fio' in i:
                result = self.encode_single_sample(image,ConvertionType=1)

            pole = result#''
            """
            for k in range(len(result)):
                pole = pole + ' ' + str(result[k][1])
            """
            if pole:
                pole = pole.strip()
                d[i.split('.', 1)[0]] = pole.upper().strip()
        data['inn'].append(d)
        return data['inn']

    def detect_inn(self, photo):
        pole = ['fio', 'inn']
        croped = self.get_crop_inn(photo)

        if croped != '':
            img, detect = self.yolo_5_inn(croped)

            for l in detect:
                x = l[1]
                y = l[2]
                w = l[3]
                h = l[4]

            obl = self.oblasty_yolo_5_inn(img, detect)
            rec = self.recognition_slovar_inn(obl)
            key = list(rec[0].keys())
            value = list(rec[0].values())
            if set(key) == set(pole):
                if '' in value:
                    flag = 1
                else:
                    flag = 0
            else:
                flag = 1
            return rec[0], flag
        else:
            rec = {}
            return rec, 1


class Snils:
    """
    def __init__(self):
        self.reader = easyocr.Reader(['ru'],
                        model_storage_directory='EasyOCR/model',
                        user_network_directory='EasyOCR/user_network',
                        recog_network='custom_example',
                        gpu=False)

        self.model_round = torch.hub.load('yolov5_master', 'custom', path='yolo5/snils_rotation.pt', source='local')
        self.model_detect = torch.hub.load('yolov5_master', 'custom', path='yolo5/snils_detect.pt', source='local')
        self.model_numbers = torch.hub.load('yolov5_master', 'custom', path='yolo5/yolov5m.pt', source='local')
    """
    def __init__(self):
        self.ModelFolder=''#"OCRModels/"
        self.Model1=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_numbers_v2.h5')
        self.Model2=load_model(self.ModelFolder+'OCRModels/Model_w740_h64_zeroz1312-740_dropout_v2.h5')
        self.model_round = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/snils_rotation.pt', source='local')
        self.model_detect = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/snils_detect.pt', source='local')
        self.model_numbers = torch.hub.load(self.ModelFolder+'yolov5_master', 'custom', path=self.ModelFolder+'yolo5/yolov5m.pt', source='local')
        self.Model1_2_img_width = 1312  #1148  # 750
        self.Model1_2_img_height = 64  #164   # 64
        with open(self.ModelFolder+"OCRModels/characters_letters.pickle", 'rb') as f: Model1_2_characters= pickle.load(f)
        char_to_num = keras.layers.StringLookup(vocabulary=list(Model1_2_characters), mask_token=None)
        self.num_to_char = keras.layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)
        self.MaxLenLabel_1_2=80
                                                   
    def encode_single_sample(self,img,ConvertionType=0):#type1=0 - Numbers  type=1 - Text
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = np.expand_dims(img,axis=-1)
	
        img1 = img.copy()
        img1 = tf.image.convert_image_dtype(img1, tf.float32)
        img1 = tf.image.resize(img1, [self.Model1_2_img_height, 740])
        ones = tf.ones((self.Model1_2_img_height, (self.Model1_2_img_width - 740), 1), dtype=tf.float32)
        img1 = tf.concat([img1, ones], 1)
        img1 = tf.transpose(img1, perm=[1, 0, 2])
        if(ConvertionType==0):
            Prediction=self.Model1.predict(np.expand_dims(img1,0),verbose=0)#np.expand_dims(ImArr,0)
        elif(ConvertionType==1):
            Prediction=self.Model2.predict(np.expand_dims(img1,0),verbose=0)
			
        input_len = np.ones(Prediction.shape[0]) * Prediction.shape[1]
        # Use greedy search. For complex tasks, you can use beam search
        results = keras.backend.ctc_decode(Prediction, input_length=input_len, greedy=True)[0][0][:,:self.MaxLenLabel_1_2]
        output_text=tf.strings.reduce_join(self.num_to_char(results[0])).numpy().decode("utf-8")
        output_text=output_text.replace("[UNK]","")
        #print(output_text)
        BreaksEnd=False
        BreaksEndPos=0
        for i in range(-1,-len(output_text),-1):
            if(output_text[i]!=" " and BreaksEndPos==0):
                BreaksEnd=True
                BreaksEndPos=i
                break
            #print(BreaksEndPos)
        if(BreaksEndPos<-1):
            output_text=output_text[:BreaksEndPos+1]
        #print("output_text",output_text)
        return output_text


    def rotate_image(self, mat, angle, point):
        height, width = mat.shape[:2]
        image_center = (width/2, height/2)
        rotation_mat = cv2.getRotationMatrix2D(image_center, angle, 1.)

        # rotation calculates the cos and sin, taking absolutes of those.
        abs_cos = abs(rotation_mat[0,0])
        abs_sin = abs(rotation_mat[0,1])

        # find the new width and height bounds
        bound_w = int(height * abs_sin + width * abs_cos)
        bound_h = int(height * abs_cos + width * abs_sin)

        # subtract old image center (bringing image back to origo) and adding the new image center coordinates
        rotation_mat[0, 2] += bound_w/2 - image_center[0]
        rotation_mat[1, 2] += bound_h/2 - image_center[1]

        # rotate image with the new bounds and translated rotation matrix
        rotated_mat = cv2.warpAffine(mat, rotation_mat, (bound_w, bound_h))
        return rotated_mat

    def get_angle_rotation(self, centre, point, target_angle):
        new_point =(point[0] - centre[0], point[1] - centre[1])
        a,b = new_point[0], new_point[1]
        res = math.atan2(b,a)
        if (res < 0) :
            res += 2 * math.pi
        return (math.degrees(res)+target_angle) % 360

    def get_image_after_rotation(self, img):
        results = self.model_round(img)
        ('get_image_after_rotation',results)
        pd = results.pandas().xyxy[0]
        pd = pd.assign(centre_x = pd.xmin + (pd.xmax-pd.xmin)/2)
        pd = pd.assign(centre_y = pd.ymin + (pd.ymax-pd.ymin)/2)

        tmp = pd.loc[pd['name']=='Svidetelstvo_test']

        N, V = None, None
        for index, row in tmp.iterrows():
            N = (row['centre_x'], row['centre_y'])
            break
        #получим координаты верха, там где печать
        tmp = pd.loc[pd['name']=='Svidetelstvo_train']
        for index, row in tmp.iterrows():
            V = (row['centre_x'], row['centre_y'])
            break
        if N == None or V == None: #похоже там нет нужных нам строк
            return img

        angle = self.get_angle_rotation(N, V, 180)

        img = self.rotate_image(img, angle, N)  #вращаем той процедурой, что выше
        return img

    def crop_img(self, img):
        results = self.model_round(img)
        pd = results.pandas().xyxy[0]
        try:
        #определяем координаты вырезки
            x1 =int(pd.xmin.min())
            x2 = int(pd.xmax.max())

            y1 = int(pd.ymin.min())
            y2 = int(pd.ymax.max())

            img = img[y1:y2,x1:x2]
        except Exception:
            print(pd)
        return img

    def get_crop(self, file):
        image = cv2.imread(file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image = self.get_image_after_rotation(image)
        image = self.get_image_after_rotation(image) #второй подряд поворот еще лучше выравнивает

        return image

    def zero(self,n):
        return n * (n > 0)

    def yolo_5_snils(self, img):
        results = self.model_detect(img)
        df = results.pandas().xyxy[0]
        df = df.drop(np.where(df['confidence'] < 0.6)[0])
        ob = pd.DataFrame()
        ob['class'] = df['name']
        ob['x'] = df['xmin']
        ob['y'] = df['ymin']
        ob['w'] = df['xmax']-df['xmin']
        ob['h'] = df['ymax']-df['ymin']
        oblasty = ob.values.tolist()
        return img, oblasty

    def oblasty_yolo_5_snils(self, image, box):
        oblasty = {}
        spissok = sorted(box, reverse=False, key=lambda x: x[2])
        for l in spissok:
            cat = l[0]
            y = int(l[2])
            x = int(l[1])
            h = int(l[4])
            w = int(l[3])
            ob = cat
            oblasty[ob] = image[self.zero(y - math.ceil(h * 0.1)):y + math.ceil(h * 1.1),
                      self.zero(x - math.ceil(w * 0.03)):x + math.ceil(w * 1.03)]
        return oblasty

    def Intersection(self, boxA, boxB):
        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
        return interArea

    def IoU(self, boxA, boxB):
        # compute the area of intersection rectangle
        interArea = self.Intersection(boxA, boxB)

        # compute the area of both the prediction and ground-truth rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

        # calculation iou
        iou = interArea / float(boxAArea + boxBArea - interArea)

        # return the intersection over union value
        return iou

    def numbers_detect(self, img):

        image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        result = self.model_numbers(image)

        final_res = result.pandas().xyxy[0]
        result = result.pandas().xyxy[0]

        # delete intersection objects
        for obj1 in range(len(result)):
            for obj2 in range(len(result)):

                boxA = result.iloc[obj1, :4].values
                boxB = result.iloc[obj2, :4].values

                if boxA.all() != boxB.all():
                    if self.IoU(boxA, boxB) > 0.2:
                        if result.iloc[obj1, 4] > result.iloc[obj2, 4]:
                            final_res = final_res[final_res.xmin != result.iloc[obj2, 0]]

                        else:
                            final_res = final_res[final_res.xmin != result.iloc[obj1, 0]]
        return final_res


    def recognition_slovar_snils(self, oblasty):
        data = {}
        data['snils'] = []
        d = {}
        fio=''
        for i, v in oblasty.items():
            image = cv2.cvtColor(v, cv2.COLOR_BGR2RGB)

            # plt.imshow(image)
            # plt.show()

            if 'number_strah' in i:
                # result = self.reader.readtext(image, allowlist='0123456789- ')
                """
                numbs = self.numbers_detect(image).sort_values(by=['xmin']).iloc[:, 5].values
                res_numbs = ''.join(str(e) for e in numbs)
                res_numbs = f'{res_numbs[:3]}-{res_numbs[3:6]}-{res_numbs[6:9]}_{res_numbs[-2:]}'
                result = [([], f'{res_numbs}', 0.0)]
                """
                result = self.encode_single_sample(image, ConvertionType=0)
            elif 'fio' in i:
                result = self.encode_single_sample(image, ConvertionType=1)
                                                   
            pole = result#''
            """
            for k in range(len(result)):
                pole = pole + ' ' + str(result[k][1]).replace(' ', '').replace('_', ' ')
            if pole:
                pole = pole.strip()
                if 'fio' in i:
                    fio = fio + ' ' + pole.upper().strip()
                else:
                    d[i.split('.', 1)[0]] = pole.upper().strip()
            """
            if 'fio' in i:
                    fio = fio + ' ' + pole.upper().strip()
            else:
                    d[i.split('.', 1)[0]] = pole.upper().strip()
                    
                    
        d['fio'] = fio.strip()
        data['snils'].append(d)
        return data['snils']

    def detect_snils(self,photo):
        pole = ['number_strah','fio1','fio2','fio3']

        croped = self.get_crop(photo)
        if croped != '':
            img, detect = self.yolo_5_snils(croped)
            obl = self.oblasty_yolo_5_snils(img, detect)
            rec = self.recognition_slovar_snils(obl)
            key = list(rec[0].keys())
            value = list(rec[0].values())
            if set(key) == set(pole):
                if '' in value:
                    flag = 1
                else:
                    flag = 0
            else:
                flag = 1
            return rec[0], flag
        else:
            rec = {}
            return rec, 1



In [4]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble

from drive.MyDrive.AI.Proekts.pasport.ClassAssemble.passport_class import Passport,INN,Snils

!cp -R /content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/yolo5 yolo5
!cp -R /content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/yolov5_master yolov5_master
!cp -R /content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/OCRModels OCRModels

psprt = Passport()
Inn1 = INN()
Snils1 = Snils()


Mounted at /content/drive
OCRModels	   __pycache__	     yolo5
passport_class.py  requirements.txt  yolov5_master


YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 


[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/pyparsing-3.0.9.dist-info/METADATA'


Model summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU



[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/cycler-0.11.0.dist-info/METADATA'


Fusing layers... 
Model summary: 322 layers, 86254162 parameters, 0 gradients, 204.0 GFLOPs
Adding AutoShape... 
No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 


[Errno 2] No such file or directory: '/usr/local/lib/python3.10/dist-packages/idna-3.4.dist-info/METADATA'


Model summary: 290 layers, 20865057 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 290 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 322 layers, 86233975 parameters, 0 gradients, 203.9 GFLOPs
Adding AutoShape... 
No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
No training configuration found in the save file, so the model was *not* compiled. Compile it manually.
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 212 layers, 20856975 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-1 Python-3.10.11 torch-2.0.0+cu118 CPU

Fusing layers... 
Model summary: 267 layers, 46124433 parameters, 0 gradients, 107.7 GFLOPs
Adding AutoShape... 
YOLOv5 🚀 2023-5-1 Python

In [5]:
folder_path = '/content/drive/MyDrive/AI/Proekts/pasport/test_pass_passport/test_pass'
results=[]
for file_name in os.listdir(folder_path):
    if file_name.endswith('.jpeg'):
        file_path = os.path.join(folder_path, file_name)
        data1=psprt.detect_passport(file_path)
        results.append((file_name, data1))
  
results

/content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/passport_class.py:322: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if croped != '':
/content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/passport_class.py:322: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if croped != '':
/content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/passport_class.py:322: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if croped != '':
/content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/passport_class.py:322: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if croped != '':
/content/drive/MyDrive/AI/Proekts/pasport/ClassAssemble/passport_class.py:322: FutureWarning: elementwis

[('27f2fdc8b1ffc41d481b57ab8b568917.jpeg',
  ({'date_of_issue': '06.07.2022',
    'unit_code': '630-027',
    'surname': 'ДЖУМАЕВ',
    'first_name': 'АМЕОМИДДИН',
    'patronymic': 'САЛОМУДИНОВИЧ',
    'date_of_birth': '30.04.1990',
    'gender': 'МУЖ.',
    'issued_by_whom': 'ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ',
    'place_of_birth': 'С. АЙНИ КУМСАНГИРСКИЙ Р-М КУРГАН-ТЮБИНСКАЯ ОБЛ. РЕСП. ТАДЖИКИСТАН',
    'series_and_number': '36 22 110830'},
   0)),
 ('3710bc70ed198d96abb3f7e8e373276c.jpeg',
  ({'unit_code': '502-015',
    'date_of_issue': '15.02.2002',
    'surname': 'ОПРЯ',
    'first_name': 'ДМИТРИЙ',
    'patronymic': 'СТЕПАНОВИЧ',
    'gender': 'МУЖ.',
    'date_of_birth': '07.08.1979',
    'issued_by_whom': 'ЛОБНЕНСКИМ ОВД МЫТИШИНСКОГО УВД МОСКОВСКОЙ ОБЛАСТИ',
    'place_of_birth': 'Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.',
    'series_and_number': 'поле распознано не полностью46024527213'},
   1)),
 ('3672a1e729a3e5371b9ec45c5a91c297.jpeg',
  ({'date_of_issue': '28.09.2002',
    'unit_code'

In [12]:
df_data = []
for tup in results:
    row_dict = {}
    row_dict['file_name'] = tup[0]
    for k, v in tup[1][0].items():
        row_dict[k] = v
    df_data.append(row_dict)
df1 = pd.DataFrame(df_data)
df1 = df1.sort_values(by='file_name').reset_index(drop=True)
df1

,file_name,date_of_issue,unit_code,surname,first_name,...,date_of_birth,gender,issued_by_whom,place_of_birth,series_and_number
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,02.07.2013,380-003,НЫРКО,АНАСТАСИЯ,...,07.05.1991,ЖЕН.,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК,25 13 856830
1,177de6d27597c51ac801fe8bab8e0931.jpeg,13.01.2012,230-029,БАТИЩЕВ,ВЯЧЕСЛАВ,...,19.12.1991,МУЖ.,ОФМС РОССИИ ПО КРАСНОДАРСКОМУ КРАЮ В БЕЛОГЛИНС...,ГОР. МАЙКОП РЕСП. АДЫГЕЯ,03 11 968450
2,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,06.07.2022,630-027,ДЖУМАЕВ,АМЕОМИДДИН,...,30.04.1990,МУЖ.,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-М КУРГАН-ТЮБИНСКАЯ ОБЛ...,36 22 110830
3,3672a1e729a3e5371b9ec45c5a91c297.jpeg,28.09.2002,632-010,РУБАНОВ,ДМИТРИЙ,...,20.11.1979,МУЖ.,ОКТЯБРЬСКИМ РОВД ГОРОДА САМАРЫ,Г. КУЙБЫШЕВ,36 02 551539
4,3710bc70ed198d96abb3f7e8e373276c.jpeg,15.02.2002,502-015,ОПРЯ,ДМИТРИЙ,...,07.08.1979,МУЖ.,ЛОБНЕНСКИМ ОВД МЫТИШИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.,поле распознано не полностью46024527213
5,44ab4177f18cd5dd8a898c852c5fd2c4.jpeg,24.07.2009,550-038,ГУДЫМ,МАКСИМ,...,30.09.1984,МУЖ.,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ОМСКОЙ ОБЛАСТИ В ЧЕР...,Г. МИМХАЙЛОВКА ФЕРЛАКСКОГО Р-НА ОМСКОЙ ОБЛ.,52 09 789231
6,5139349d9409170acd30abcf996f09cf.jpeg,15.06.2015,640-011,БАЛТАДЖЯН,АРКАДИЙ,...,04.07.1956,МУЖ.,ОТДЕЛОМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛАСТИ В Г...,ГОР. ТБИЛИСИ Г. ССР,63 15 160426
7,582b6e51f655e69e33dcbb6049cae588.jpeg,01.04.2002,162-013,БАШАН,АЛЕКСЕЙ,...,26.07.1981,МУЖ.,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ,92 02 983452
8,68418534-4c5f-43db-8277-37b9596d31ae.jpeg,07.12.2017,360-004,КУРАСОВ,АЛЕКСАНДР,...,06.04.1971,МУЖ.,ОТДЕЛОМ УФМС РОССИН ПО ВОРОНЕЖСКОЙ ОБЛАСТИ В К...,С. В-ПЕСКИ ЭРТИЛЬСКОГО Р-НА ВОРОНЕЖСКОЙ ОБЛ.,20 17 158065
9,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,25.06.2007,780-001,ЕГОРОВ,ВАСИЛИЙ,...,20.04.1987,МУЖ.,ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-НЕТЕРБУРГУ И...,ГОР. ЛЕНИНГРАД,40 07 187829


In [22]:
df = pd.read_excel('/content/drive/MyDrive/AI/Proekts/pasport/test_pass_passport/test_pass/test_pass_labels.xlsx')
df.fillna('', inplace=True)
df['issued_by_whom'] = df['issued_by_whom_0'].apply(lambda x: str(x)) + ' ' + df['issued_by_whom_1'].apply(lambda x: str(x)) + ' ' + df['issued_by_whom_2'].apply(lambda x: str(x))
df['place_of_birth'] = df['place_of_birth_0'].apply(lambda x: str(x)) + ' ' + df['place_of_birth_1'].apply(lambda x: str(x)) + ' ' + df['place_of_birth_2'].apply(lambda x: str(x))
df.rename(columns={'series_and_number_1': 'series_and_number'}, inplace=True)
df2 = df.drop(columns=['issued_by_whom_0', 'issued_by_whom_1', 'issued_by_whom_2', 'place_of_birth_0', 'place_of_birth_1', 'place_of_birth_2', 'series_and_number_2'])
df2

,file_name,date_of_issue,unit_code,surname,first_name,...,date_of_birth,gender,series_and_number,issued_by_whom,place_of_birth
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,02.07.2013,380-003,НЫРКО,АНАСТАСИЯ,...,07.05.1991,ЖЕН.,25 13 856830,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК
1,d099912b-fa46-49ac-8f7b-0f660b184ce6.jpeg,18.08.2010,640-036,РОЩИН,СЕРГЕЙ,...,11.02.1990,МУЖ.,63 10 509663,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛ. В Г...,С. ПОТЬМА РТИЩЕВСКИЙ Р-ОН САРАТОВСКАЯ ОБЛ.
2,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,25.06.2007,780-001,ЕГОРОВ,ВАСИЛИЙ,...,20.04.1987,МУЖ.,40 07 187829,ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-ПЕТЕРБУРГУ И...,ГОР. ЛЕНИНГРАД
3,de6102e3-2a98-4898-90b2-18f30ba3e912.jpeg,09.08.2017,690-027,КОВАЛЁВ,ЕВГЕНИЙ,...,13.06.1992,МУЖ.,28 17 540959,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ТВЕРСКОЙ ОБЛАСТИ В О...,ГОР. ОСТАШКОВ ТВЕРСКАЯ ОБЛ.
4,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,06.07.2022,630-027,ДЖУМАЕВ,АХРОМИДДИН,...,30.04.1990,МУЖ.,36 22 110830,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-Н КУРГАН-ТЮБИНСКАЯ ОБЛ...
5,3710bc70ed198d96abb3f7e8e373276c.jpeg,15.02.2002,502-015,ОПРЯ,ДМИТРИЙ,...,07.08.1979,МУЖ.,46 02 527213,ЛОБНЕНСКИМ ОВД МЫТИЩИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.
6,c10be491ce2c9f2a29747b88ad232b08.jpeg,20.09.2007,770-074,КАЛИНКИН,АЛЕКСАНДР,...,07.11.1984,МУЖ.,45 09 194829,ОТДЕЛЕНИЕМ ПО РАЙОНУ ФИЛЕВСКИЙ ПАРК ОУФМС РОСС...,ГОРОД МОСКВА
7,985317e9a09699992b2edcfd5e27aff5.jpeg,31.05.2022,460-012,ДЗЕКИЛВ,АЛЕКСЕЙ,...,30.04.1977,МУЖ.,38 22 393737,УМВД РОССИИ ПО КУРСКОЙ ОБЛАСТИ,Г. ЖЕЛЕЗНОГОРСК КУРСКАЯ ОБЛ.
8,582b6e51f655e69e33dcbb6049cae588.jpeg,01.04.2002,162-013,БАШАН,АЛЕКСЕЙ,...,26.07.1981,МУЖ.,92 02 983452,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ
9,c20e6682-54c4-4c26-949b-cb5d5fa8ea39.jpeg,25.12.2012,660-009,МОШКИН,ИГОРЬ,...,19.04.1967,МУЖ.,65 12 538322,ОУФМС РОССИИ ПО СВЕРДЛОВСКОЙ ОБЛ. В ЧКАЛОВСКОМ...,ГОР. СВЕРДЛОВСК


In [33]:
df3 = df2.reindex(columns=df1.columns)
df3 = df3.sort_values(by='file_name').reset_index(drop=True)
df3['issued_by_whom'] = df3['issued_by_whom'].astype(str)
df3['issued_by_whom'] = df3['issued_by_whom'].str.strip()
df3['place_of_birth'] = df3['place_of_birth'].astype(str)
df3['place_of_birth'] = df3['place_of_birth'].str.strip()
df3

,file_name,date_of_issue,unit_code,surname,first_name,...,date_of_birth,gender,issued_by_whom,place_of_birth,series_and_number
0,0c5cd9f2-96c9-43ed-9008-f6de3b1a8a8c.jpeg,02.07.2013,380-003,НЫРКО,АНАСТАСИЯ,...,07.05.1991,ЖЕН.,ОТДЕЛОМ УФМС РОССИИ ПО ИРКУТСКОЙ ОБЛАСТИ В ОКТ...,ГОР. ИРКУТСК,25 13 856830
1,177de6d27597c51ac801fe8bab8e0931.jpeg,13.01.2012,230-029,БАТИЩЕВ,ВЯЧЕСЛАВ,...,19.12.1991,МУЖ.,ОУФМС РОССИИ ПО КРАСНОДАРСКОМУ КРАЮ В БЕЛОГЛИН...,ГОР. МАЙКОП РЕСП. АДЫГЕЯ,03 11 968450
2,27f2fdc8b1ffc41d481b57ab8b568917.jpeg,06.07.2022,630-027,ДЖУМАЕВ,АХРОМИДДИН,...,30.04.1990,МУЖ.,ГУ МВД РОССИИ ПО САМАРСКОЙ ОБЛАСТИ,С. АЙНИ КУМСАНГИРСКИЙ Р-Н КУРГАН-ТЮБИНСКАЯ ОБЛ...,36 22 110830
3,3672a1e729a3e5371b9ec45c5a91c297.jpeg,28.09.2002,632-010,РУБАНОВ,ДМИТРИЙ,...,20.11.1979,МУЖ,ОКТЯБРЬСКИМ РОВД ГОРОДА САМАРЫ,Г. КУЙБЫШЕВ,36 02 551539
4,3710bc70ed198d96abb3f7e8e373276c.jpeg,15.02.2002,502-015,ОПРЯ,ДМИТРИЙ,...,07.08.1979,МУЖ.,ЛОБНЕНСКИМ ОВД МЫТИЩИНСКОГО УВД МОСКОВСКОЙ ОБЛ...,Г. ЛОБНЯ МОСКОВСКОЙ ОБЛ.,46 02 527213
5,44ab4177f18cd5dd8a898c852c5fd2c4.jpeg,24.07.2009,550-038,ГУДЫМ,МАКСИМ,...,30.09.1984,МУЖ.,ОТДЕЛЕНИЕМ УФМС РОССИИ ПО ОМСКОЙ ОБЛАСТИ В ЧЕР...,С. МИХАЙЛОВКА ЧЕРЛАКСКОГО Р-НА ОМСКОЙ ОБЛ.,52 09 789231
6,5139349d9409170acd30abcf996f09cf.jpeg,15.06.2015,640-011,БАЛТАДЖЯН,АРКАДИЙ,...,04.07.1956,МУЖ.,ОТДЕЛОМ УФМС РОССИИ ПО САРАТОВСКОЙ ОБЛАСТИ В Г...,ГОР. ТБИЛИСИ Г. ССР,63 15 160426
7,582b6e51f655e69e33dcbb6049cae588.jpeg,01.04.2002,162-013,БАШАН,АЛЕКСЕЙ,...,26.07.1981,МУЖ.,ЭЛЕКТРОТЕХНИЧЕСКИМ ОВД ГОР. НАБЕРЕЖНЫЕ ЧЕЛНЫ Р...,ГОР. КАЗАНЬ,92 02 983452
8,68418534-4c5f-43db-8277-37b9596d31ae.jpeg,07.12.2017,360-004,КУРАСОВ,АЛЕКСАНДР,...,06.04.1971,МУЖ.,ОТДЕЛОМ УФМС РОССИИ ПО ВОРОНЕЖСКОЙ ОБЛАСТИ В К...,С. Б-ПЕСКИ ЭРТИЛЬСКОГО Р-НА ВОРОНЕЖСКОЙ ОБЛ.,20 17 158065
9,7d8c2607-7bd0-47c9-8011-e035a6f34dd4.jpeg,25.06.2007,780-001,ЕГОРОВ,ВАСИЛИЙ,...,20.04.1987,МУЖ.,ТП №1 ОТДЕЛА УФМС РОССИИ ПО САНКТ-ПЕТЕРБУРГУ И...,ГОР. ЛЕНИНГРАД,40 07 187829


In [52]:
df4 = df1.eq(df3)
df4 = df4.replace({True: 1, False: 0})
df4 = df4.astype(int)
df4['%'] = df4.iloc[:, 1:].sum(axis=1)/10*100
#sum_row = df4.sum(axis=0)
#sum_row
#df4 = df.append(sum_row, ignore_index=True)/21*100
#df4['result'] = df.apply(lambda row: 0 if 0 in row.values[1:] else 1, axis=1).astype(int)
sum_row = pd.DataFrame(df4.iloc[:, :-1].sum()).T/21*100
sum_row.columns = df4.columns[:-1]
df4 = df4.append(sum_row, ignore_index=True)
df4


<ipython-input-52-78992aeb61f9>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df4 = df4.append(sum_row, ignore_index=True)


,file_name,date_of_issue,unit_code,surname,first_name,...,gender,issued_by_whom,place_of_birth,series_and_number,%
0,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,1.000000,1.000000,1.00000,100.0
1,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,0.000000,1.000000,1.00000,90.0
2,1.0,1.000000,1.0,1.00000,0.000000,...,1.00000,1.000000,0.000000,1.00000,80.0
3,1.0,1.000000,1.0,1.00000,1.000000,...,0.00000,1.000000,1.000000,1.00000,90.0
4,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,0.000000,1.000000,0.00000,80.0
5,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,1.000000,0.000000,1.00000,90.0
6,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,1.000000,1.000000,1.00000,100.0
7,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,1.000000,1.000000,1.00000,100.0
8,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,0.000000,0.000000,1.00000,80.0
9,1.0,1.000000,1.0,1.00000,1.000000,...,1.00000,0.000000,1.000000,1.00000,90.0
